In [ ]:
#pip install dask

In [ ]:
#import dask.dataframe as dd
import numpy as np
import pandas as pd

#from dask.diagnostics import ProgressBar
#ProgressBar().register()

import nltk
import re
import string
import time
import csv
from tqdm import tqdm

nltk.download('stopwords')
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /home/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
## If you are using the data by mounting the google drive, use the following :
# from google.colab import drive
# drive.mount('/content/gdrive',  force_remount=False)

# %cd /content/gdrive/MyDrive/FinalThesis/Bitcoin_Datasets/

In [ ]:
#Load Twitter Dataset
start = time.time()
tweet_raw= pd.read_csv("Bitcoin_tweets.csv", encoding = 'utf8')
t = time.time() - start
print('Time taken to load Twitter Data:',t)

/tmp/ipykernel_259/4093008141.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  tweet_raw= pd.read_csv("Bitcoin_tweets.csv", encoding = 'utf8')


Time taken to load Twitter Data: 10.306974649429321


In [ ]:
text_data = tweet_raw.drop(columns= ['user_name','user_location','user_description','user_created','user_followers','user_friends','user_favourites','user_verified','is_retweet','hashtags','source'])

In [ ]:
text_data.dtypes

date    float64
text     object
dtype: object

In [ ]:
text_data['date_time'] = pd.to_datetime(text_data['date'], unit='d', origin='1899-12-30')
text_data  = text_data.astype({"text": "string"})

In [ ]:
text_data.dtypes

date                float64
text                 string
date_time    datetime64[ns]
dtype: object

In [ ]:
text_data = text_data.drop('date', axis = 1)

In [ ]:
text_data.head(10)

,text,date_time
0,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04.000003840
1,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48.000002560
2,"Guys evening, I have read this article about B...",2021-02-10 23:54:48.000001280
3,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33.000002048
4,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06.000002048
5,💹 Trade #Crypto on #Binance 📌 Enjoy #Cashbac...,2021-02-10 23:53:29.999998720
6,&lt;'fire' &amp; 'man'&gt; #Bitcoin #Crypto #B...,2021-02-10 23:53:16.999995648
7,🔄 Prices update in $EUR (1 hour): $BTC - 37...,2021-02-10 23:52:42.000003840
8,#BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...,2021-02-10 23:52:25.000000512
9,.@Tesla’s #bitcoin investment is revolutionary...,2021-02-10 23:52:07.999996416


# Data Cleaning

a)	Convert tweets to lowercase. D

b)	Expand words with contractions.

c)	Removed duplicate tweets based on unique id. NA

d)	Remove stopwords from the tweet. D

e)	Remove punctuation and special characters. D

f)	Remove user mentions that are a word starting with the @ sign. D

g)	Replace over 2 dots and multiple spaces with a single space. D

h)	Remove dollar symbols from the text like $BTC and $Bitcoin. D

i)	Remove the hashtag sign from the text. D

j)	Remove non-English tweets. D

k)	Remove URLs from tweets. D
  

In [ ]:
def clean_text_data(tweet):
  # Transform to lower case
  tweet = tweet.lower()
  #Remove url, square and round brackets, mentions
  tweet = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([@]\S+)", "",tweet)
  #Remove numbers
  tweet = re.sub(r'\d', '', tweet) #reconsider this step
  #Remove stop words
  tweet = ' '.join([word for word in tweet.split() if word not in cachedStopWords  and  word.isascii()])
  #Remove Punctuations
  tweet = tweet.translate(str.maketrans('', '','!"#$%&.\'()*+,-/:;<=>?@[\]^_`{|}~'))
  #Remove extra spaces
  tweet = re.sub('[\s]+', ' ', tweet)

  return tweet

In [ ]:
#print (clean_text_data('Blue Ridge Bank shares ------- halted by NYSE  ~ after #bitcoin .......ATM announcement https://t.co/xaaZmaJKiV @MyBlueRidgeBank… https://t.co/sgBxMkP1SI'))

In [ ]:
#text_data = text_data.head(20000)

In [ ]:
text_data.shape

(1048570, 2)

In [ ]:
text_data = text_data.dropna()

In [ ]:
text_data.shape

(1048570, 2)

In [ ]:
start = time.time()
tqdm.pandas(desc="Cleaning Data........")
text_data["clean_text"] = text_data.text.progress_apply(lambda x: clean_text_data(x))
t = time.time() - start
print("Time taken to clean textual data:",t)

Cleaning Data........: 100%|██████████| 1048570/1048570 [01:17<00:00, 13449.04it/s]

Time taken to clean textual data: 78.06545281410217


In [ ]:
text_data.head(10)

,date_time,clean_text
0,2021-02-10 23:59:04.000003840,blue ridge bank shares halted nyse bitcoin atm...
1,2021-02-10 23:58:48.000002560,today thats thursday take friend btc wallet se...
2,2021-02-10 23:54:48.000001280,guys evening read article btc would like share
3,2021-02-10 23:54:33.000002048,btc big chance billion price bitcoin fx btc cr...
4,2021-02-10 23:54:06.000002048,network secured nodes today soon biggest bears...
5,2021-02-10 23:53:29.999998720,trade crypto binance enjoy cashback trading fe...
6,2021-02-10 23:53:16.999995648,ltfire amp mangt bitcoin crypto btc
7,2021-02-10 23:52:42.000003840,prices update eur btc eth xrp
8,2021-02-10 23:52:25.000000512,btc bitcoin ethereum eth crypto cryptotrading ...
9,2021-02-10 23:52:07.999996416,bitcoin investment revolutionary crypto firms...


In [ ]:
text_data = text_data[['date_time','clean_text']]

In [ ]:
clean_data = text_data.dropna()

In [ ]:
clean_data.shape

(1048570, 2)

In [ ]:
clean_data['date_time'] = clean_data['date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
clean_data.head(10)

,date_time,clean_text
0,2021-02-10 23:59:04,blue ridge bank shares halted nyse bitcoin atm...
1,2021-02-10 23:58:48,today thats thursday take friend btc wallet se...
2,2021-02-10 23:54:48,guys evening read article btc would like share
3,2021-02-10 23:54:33,btc big chance billion price bitcoin fx btc cr...
4,2021-02-10 23:54:06,network secured nodes today soon biggest bears...
5,2021-02-10 23:53:29,trade crypto binance enjoy cashback trading fe...
6,2021-02-10 23:53:16,ltfire amp mangt bitcoin crypto btc
7,2021-02-10 23:52:42,prices update eur btc eth xrp
8,2021-02-10 23:52:25,btc bitcoin ethereum eth crypto cryptotrading ...
9,2021-02-10 23:52:07,bitcoin investment revolutionary crypto firms...


In [ ]:
today_date = time.strftime("%d-%m-%Y")
excel_filename = "clean_data_"+today_date +".csv"
print(excel_filename)
clean_data.to_csv(excel_filename, index=False, header=True)

clean_data_20-01-2024.csv
